In [1]:
import os
import torch
import torchvision
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import pickle

from fastai.vision.all import *

from PIL import Image
from torchvision.transforms import ToTensor
from responsibleai_vision.common.constants import ImageColumns
from tqdm import tqdm

BATCH_SIZE = 4
DATASET_NAME = "CIFAR"

dataset = torchvision.datasets.CIFAR10(root="data", train=False, download=True, transform=ToTensor())
data_loader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

def convert_torch_model(model, df, train=False):
    dls = ImageDataLoaders.from_df(df, path='./')
    learn = Learner(dls, model)
    
    if train:
        xb,yb = learn.dls.one_batch()
        init_loss = learn.loss_func(learn.model(xb), yb)
        learn.fit_one_cycle(10)
        xb,yb = learn.dls.one_batch()
        final_loss = learn.loss_func(learn.model(xb), yb)
        print(final_loss)
    
    return learn
    
        
def convert_torch_data(data_loader):
    
    data = pd.DataFrame(columns=[ImageColumns.IMAGE.value,
                                 ImageColumns.LABEL.value])

    cnts = {}
    data_path = "data/" + DATASET_NAME + "/" 
    
    for batch in tqdm(data_loader):
        
        images, labels = batch
        for i in range(BATCH_SIZE):
            img, label = images[i], labels[i]
            label = str(label.numpy())
            img = img.numpy()
            img = np.transpose(img, (1, 2, 0))
            img = Image.fromarray((img * 255).astype(np.uint8))

            if label not in cnts:
                os.makedirs(data_path + label, exist_ok=True)
                cnts[label] = 1
            else:
                cnts[label] += 1

            img_path = data_path + label + "/class_" + label + "_img_" + str(cnts[label]) + ".jpeg"
            img.save(img_path)
            item = pd.DataFrame(data={ImageColumns.IMAGE.value: [img_path], 
                                      ImageColumns.LABEL.value: [label]})
            data = pd.concat([data, item], ignore_index=True)
                
    return data

data = convert_torch_data(data_loader)

test_data = data
class_names = data[ImageColumns.LABEL.value].unique()

No audio backend is available.
UserConfigValidationException will be deprecated from dice_ml.utils. Please import UserConfigValidationException from raiutils.exceptions.


Files already downloaded and verified


100%|█████████████████████████████████████████████████████████████████████████████| 2500/2500 [00:19<00:00, 128.36it/s]


In [2]:
device = 'cuda'

class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = CNN().to(device)

model = convert_torch_model(model, data, train=False)

epoch,train_loss,valid_loss,time
0,2.277305,2.220090,01:25
1,2.034985,1.999362,00:22
2,1.911176,1.948487,00:19
3,1.773655,1.718257,00:23
4,1.663839,1.673275,00:16
5,1.570743,1.614541,00:15
6,1.526328,1.565496,00:18
7,1.483035,1.546891,00:21
8,1.460968,1.526650,00:19
9,1.447833,1.526867,00:16


TensorBase(1.3306, device='cuda:0', grad_fn=<AliasBackward0>)


In [3]:
from raiwidgets import ResponsibleAIDashboard
from responsibleai_vision import ModelTask, RAIVisionInsights

rai_insights = RAIVisionInsights(model, data.sample(10, random_state=42),
                                "label", task_type=ModelTask.IMAGE_CLASSIFICATION,
                                classes=class_names)
rai_insights.explainer.add()
rai_insights.error_analysis.add()
rai_insights.compute()

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 501.15it/s]



Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'heap' of function 'hq_heappop.<locals>.hq_heappop_impl'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\..\AppData\Local\anaconda3\envs\work\lib\site-packages\numba\cpython\heapq.py", line 138:

    def hq_heappop_impl(heap):
    ^


Encountered the use of a type that is scheduled for deprecation: type 'reflected list' found for argument 'seq' of function 'sequence_bool.<locals>.sequence_bool_impl'.

For more information visit https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-reflection-for-list-and-set-types

File "..\..\..\..\..\AppData\Local\anaconda3\envs\work\lib\site-packages\numba\cpython\listobj.py", line 672:
def sequence_bool(context, builder, sig, args):
    def sequence_bool_impl(seq):
    ^


Encountered the use of a type t

Error Analysis
Current Status: Generating error analysis reports.
Current Status: Finished generating error analysis reports.
Time taken: 0.0 min 0.1175010000000043 sec


In [4]:
ResponsibleAIDashboard(rai_insights)

[8, 8, 6, 1, 2, 6, 4, 9, 5, 7]
ResponsibleAI started at http://localhost:5000
